<a href="https://colab.research.google.com/github/usrini1215/occupational-trends-project/blob/master/data_cleanup_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (134 kB/s)
Reading package lists... Done


In [19]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-12-18 05:07:24--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.06MB/s    in 0.2s    

2020-12-18 05:07:24 (5.06 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [20]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [21]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://data.edd.ca.gov/resource/bcij-5wym.csv"
spark.sparkContext.addFile(url)
unemployment_rate_by_age = spark.read.csv(SparkFiles.get("bcij-5wym.csv"), sep=",", header=True, inferSchema=True)
unemployment_rate_by_age = unemployment_rate_by_age[["area_name","year","age_16_19","age_20_24","age_25_34","age_35_44","age_45_54","age_55_64","age_65"]]
# Show DataFrame
unemployment_rate_by_age.show()

+----------+----+---------+---------+---------+---------+---------+---------+------+
| area_name|year|age_16_19|age_20_24|age_25_34|age_35_44|age_45_54|age_55_64|age_65|
+----------+----+---------+---------+---------+---------+---------+---------+------+
|California|2020|    0.225|    0.152|    0.099|    0.076|    0.076|     0.08| 0.091|
|California|2020|    0.221|    0.146|    0.095|    0.071|    0.072|    0.075| 0.088|
|California|2020|    0.215|    0.138|    0.087|    0.066|    0.068|    0.069| 0.081|
|California|2020|    0.211|    0.129|    0.079|    0.059|    0.063|    0.063| 0.073|
|California|2020|      0.2|    0.118|    0.071|    0.052|    0.057|    0.058| 0.066|
|California|2020|    0.186|    0.105|    0.062|    0.045|     0.05|     0.05| 0.058|
|California|2020|    0.167|    0.091|    0.051|    0.038|    0.041|    0.043| 0.046|
|California|2020|    0.151|    0.076|    0.041|    0.029|    0.031|    0.034| 0.036|
|California|2020|    0.148|    0.073|    0.041|    0.029|    0.03

In [22]:
url="https://occupation-trends-bucket.s3-us-west-1.amazonaws.com/Local_Area_Unemployment_Statistics__LAUS_.csv"
spark.sparkContext.addFile(url)
local_area_unemployment_stats = spark.read.csv(SparkFiles.get("Local_Area_Unemployment_Statistics__LAUS_.csv"), sep=",", header=True, inferSchema=True)
local_area_unemployment_stats = local_area_unemployment_stats[["Area Name ","Year ","Month","Employment ","Unemployment ","Unemployment Rate "]]

local_area_unemployment_stats = local_area_unemployment_stats.withColumnRenamed("Area Name ", "area_name") \
        .withColumnRenamed("Year ", "year") \
        .withColumnRenamed("Month", "month") \
        .withColumnRenamed("Employment ", "employment") \
        .withColumnRenamed("Unemployment ", "unemployment") \
        .withColumnRenamed("Unemployment Rate ", "unemployment_rate")

# Show DataFrame
local_area_unemployment_stats.show()

+--------------+----+-------+----------+------------+-----------------+
|     area_name|year|  month|employment|unemployment|unemployment_rate|
+--------------+----+-------+----------+------------+-----------------+
|    California|1976|January|   8836000|      898600|            0.092|
|    California|1977|January|   9201400|      931100|            0.092|
|    California|1978|January|   9868400|      806700|            0.076|
|    California|1979|January|  10404200|      726700|            0.065|
|    California|1980|January|  10774000|      705800|            0.061|
|    California|1981|January|  10899600|      830100|            0.071|
|    California|1982|January|  10941400|     1051600|            0.088|
|    California|1983|January|  10900400|     1341800|             0.11|
|    California|1984|January|  11384000|     1026400|            0.083|
|    California|1985|January|  11936200|      939500|            0.073|
|    California|1986|January|  12241900|      894400|           

In [23]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

url="https://occupation-trends-bucket.s3-us-west-1.amazonaws.com/Occupational_Employment_Statistics__OES_.csv"
spark.sparkContext.addFile(url)
occupational_employment_stat = spark.read.csv(SparkFiles.get("Occupational_Employment_Statistics__OES_.csv"), sep=",", header=True, inferSchema=True)
occupational_employment_stat = occupational_employment_stat[["Area Type","Year","Wage Type","Occupational Title","Mean Wage"]]
occupational_employment_stat = occupational_employment_stat.filter(col("Wage Type") == "Hourly wage")

occupational_employment_stat = occupational_employment_stat.withColumnRenamed("Area Name", "area_name") \
        .withColumnRenamed("Year", "year") \
        .withColumnRenamed("Wage Type", "wage_type") \
        .withColumnRenamed("Occupational Title", "occupational_title") \
        .withColumnRenamed("Mean Wage", "mean_wage")

# Show DataFrame
occupational_employment_stat.show()

+-----------------+----+-----------+--------------------+---------+
|        Area Type|year|  wage_type|  occupational_title|mean_wage|
+-----------------+----+-----------+--------------------+---------+
|Metropolitan Area|2009|Hourly wage|Total, All Occupa...|    23.25|
|Metropolitan Area|2009|Hourly wage|Management Occupa...|    56.62|
|Metropolitan Area|2009|Hourly wage|    Chief Executives|      0.0|
|Metropolitan Area|2009|Hourly wage|General and Opera...|    63.75|
|Metropolitan Area|2009|Hourly wage|Advertising and P...|    45.99|
|Metropolitan Area|2009|Hourly wage|  Marketing Managers|    63.26|
|Metropolitan Area|2009|Hourly wage|      Sales Managers|     61.2|
|Metropolitan Area|2009|Hourly wage|Public Relations ...|     48.7|
|Metropolitan Area|2009|Hourly wage|Administrative Se...|    43.26|
|Metropolitan Area|2009|Hourly wage|Computer and Info...|    63.49|
|Metropolitan Area|2009|Hourly wage|  Financial Managers|    59.15|
|Metropolitan Area|2009|Hourly wage|Compensation

In [24]:
url="https://occupation-trends-bucket.s3-us-west-1.amazonaws.com/Current_Employment_Statistics__CES_.csv"
spark.sparkContext.addFile(url)
current_employment_stats = spark.read.csv(SparkFiles.get("Current_Employment_Statistics__CES_.csv"), sep=",", header=True, inferSchema=True)
current_employment_stats = current_employment_stats[["Area Name","Year","Industry Title","Current Employment"]]
current_employment_stats = current_employment_stats[current_employment_stats["Area Name"].contains('County')]

current_employment_stats = current_employment_stats.withColumnRenamed("Area Name", "area_name") \
        .withColumnRenamed("Year", "year") \
        .withColumnRenamed("Industry Title", "industry_title") \
        .withColumnRenamed("Current Employment", "current_employment") 


# Show DataFrame
current_employment_stats.show()

+--------------+----+--------------------+------------------+
|     area_name|year|      industry_title|current_employment|
+--------------+----+--------------------+------------------+
|Alameda County|2014|Leisure and Hospi...|             61800|
|Alameda County|2014|Health Care and S...|             95300|
|Alameda County|2014|Educational and H...|            110700|
|Alameda County|2014|        Retail Trade|             68500|
|Alameda County|2014|Administrative an...|             35200|
|Alameda County|2014|         Information|             13900|
|Alameda County|2014|     Goods Producing|            100000|
|Alameda County|2014|Arts, Entertainme...|              8800|
|Alameda County|2014|    Nondurable Goods|             21100|
|Alameda County|2014|      Other Services|             24400|
|Alameda County|2014|          Total Farm|               500|
|Alameda County|2014|Natural Resources...|             34700|
|Alameda County|2014|Financial Activities|             28300|
|Alameda

Postgres Setup

In [25]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://occupation-trends.cppwghmqrqzq.us-west-1.rds.amazonaws.com:5432/occupation-trends-db"
config = {"user":"root", 
          "password": "data1234", 
          "driver":"org.postgresql.Driver"}


In [26]:
# Write DataFrame to active_user table in RDS

unemployment_rate_by_age.write.jdbc(url=jdbc_url, table='unemployment_rate_by_age', mode=mode, properties=config)

In [29]:
# Write dataframe to billing_info table in RDS

local_area_unemployment_stats.write.jdbc(url=jdbc_url, table='local_area_unemployment_stats', mode=mode, properties=config)

In [30]:
# Write dataframe to payment_info table in RDS

occupational_employment_stat.write.jdbc(url=jdbc_url, table='occupational_employment_stat', mode=mode, properties=config)

In [14]:
# Write dataframe to payment_info table in RDS

current_employment_stats.write.jdbc(url=jdbc_url, table='current_employment_stats', mode=mode, properties=config)